# Fine Tuning do Modelo GPT-4.1 

1. Crie uma conta na Plataforma de desenvolvimento da OpenAI.
2. Insira pelo menos U$5,00 de crédito.
3. Crie uma chave API da OpenAI
4. Crie um script em Python (com auxílio de LLMs) para checar a acurácia do modelo gpt-4.1-nano frente ao problema de classificação de requisitos de sistemas nas categorias "funcional" e "não-funcional". Utilize o arquivo "dataset.jsonl" para isso. O script deverá exibir ao final a quantidade de classificações corretas e a porcentagem do total. Se você nunca programou em Python, peça auxílio de um LLM para instalar o Python e outras dependências necessárias. 
5. Realize o Fine Tuning do modelo gpt-4.1-nano com arquivo "dataset-train.jsonl". Ao criar o job de treinamento, use como sufixo o e-mail que você utilizou para se cadastrar ao classroom (e.g. "aluno@ga.ita.br").
6. Crie um script em Python (com auxílio de LLMs) para transformar o conteúdo do arquivo "dataset-test2.xlsx" numa base de dados em JSONL.
7. Una as instâncias constantes no arquivo "dataset-test.jsonl" com as instâncias oriundas do arquivo "dataset-test2.xlsx" (que você passou para JSONL no passo 6) para criar uma base de dados de teste com um número maior de instâncias. 
6. Crie outro script em Python (com auxílio de LLMs) para checar a acurácia do modelo gpt-4.1-nano "tunado", utilizando o dataset de teste que você criou no passo 7. O script deverá exibir ao final a quantidade de classificações corretas e a porcentagem do total. 
7. Entregue: 

     a) Todos os scripts. Lembre-se de documentar bem o seu código!
     
     b) O identificador único do seu modelo "tunado", nomeado com um sufixo com seu e-mail.

### Importação de Bibliotecas

In [8]:
import pandas as pd
import json
import os
from openai import OpenAI
from dotenv import load_dotenv

### Transformação do conteúdo do arquivo "dataset-test2.xlsx" numa base de dados em JSONL. (Item 6)

In [ ]:
# Caminhos
input_file = "data/xlsx/dataset-test2.xlsx"
output_file = "data/jsonl/dataset-test2.jsonl"

# Carrega Excel
df = pd.read_excel(input_file)

system_prompt = (
    "You are a Software Engineer and need to categorize requirements into "
    "'functional' or 'non-functional'. Your answer must be 'functional' or 'non-functional' only."
)

with open(output_file, "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        requirement = str(row["requirement"]).strip()
        
        # Interpreta a coluna NF
        nf_value = row["NF"]
        if str(nf_value).strip().lower() in ["1", "non-functional", "nf"]:
            label = "non-functional"
        else:
            label = "functional"
        
        example = {
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": requirement},
                {"role": "assistant", "content": label}
            ]
        }
        
        f.write(json.dumps(example, ensure_ascii=False) + "\n")

print(f"✅ Arquivo JSONL criado em: {output_file}")

✅ Arquivo JSONL criado em: data/jsonl/dataset-test2.jsonl


### Criação do arquivo "dataset-test-final.jsonl", oriundo da união da instâncias do arquivo "dataset-test.jsonl" com as instâncias do arquivo "dataset-test2.xlsx". (Item 7) 

In [6]:
# Arquivos de entrada
file1 = "data/jsonl/dataset-test2.jsonl"
file2 = "data/jsonl/dataset-test.jsonl"

# Arquivo de saída
output_file = "data/jsonl/dataset-test-final.jsonl"

with open(output_file, "w", encoding="utf-8") as outfile:
    # Copia as linhas do primeiro arquivo
    with open(file1, "r", encoding="utf-8") as infile1:
        for line in infile1:
            outfile.write(line)
    # Copia as linhas do segundo arquivo
    with open(file2, "r", encoding="utf-8") as infile2:
        for line in infile2:
            outfile.write(line)

print(f"✅ Arquivo combinado salvo em: {output_file}")

✅ Arquivo combinado salvo em: data/jsonl/dataset-test-final.jsonl


### Checando acurácia dos modelos. (Itens 4 e 8)

In [10]:
# Carregar variáveis de ambiente
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Dataset
dataset_file = "data/jsonl/dataset-test-final.jsonl"

# Modelos a comparar
models = {
    "base": "gpt-4.1-nano-2025-04-14",
    "fine_tuned": "ft:gpt-4.1-nano-2025-04-14:personal:te-251-carlos-figueiredo:CAUgUYUQ"
}

# Contadores
results = {name: {"total": 0, "correct": 0} for name in models}

with open(dataset_file, "r", encoding="utf-8") as f:
    for line in f:
        example = json.loads(line)
        messages = example["messages"]

        # expected = última resposta (assistant)
        expected = messages[-1]["content"].strip().lower()
        eval_messages = messages[:-1]  # sem o rótulo

        for name, model in models.items():
            results[name]["total"] += 1
            response = client.chat.completions.create(
                model=model,
                messages=eval_messages,
                temperature=0
            )
            predicted = response.choices[0].message.content.strip().lower()

            if predicted == expected:
                results[name]["correct"] += 1

# Exibe resultados
for name, stats in results.items():
    total = stats["total"]
    correct = stats["correct"]
    accuracy = (correct / total * 100) if total > 0 else 0
    print(f"📌 Modelo: {models[name]}")
    print(f"   ✅ Total exemplos: {total}")
    print(f"   🎯 Acertos: {correct}")
    print(f"   📊 Acurácia: {accuracy:.2f}%\n")


📌 Modelo: gpt-4.1-nano-2025-04-14
   ✅ Total exemplos: 71
   🎯 Acertos: 40
   📊 Acurácia: 56.34%

📌 Modelo: ft:gpt-4.1-nano-2025-04-14:personal:te-251-carlos-figueiredo:CAUgUYUQ
   ✅ Total exemplos: 71
   🎯 Acertos: 58
   📊 Acurácia: 81.69%

